In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
from missforest import MissForest
warnings.simplefilter("ignore")

In [2]:
df = pd.read_excel("../../data/Novice.xlsx")

In [3]:
# columns to be modified
# todo: remove all "contact" and use _
# these columns are either not relevant for match length or missing enormous amount of values (might use then for eda)
to_be_deleted = [
    "Big Employer/School Census Block Group", "Big Enrollment: Created Date", "Big Acceptance Date",
    "Big Contact: Created Date","Big Days Acceptance to Match", "Big Days Interview to Acceptance","Big Days Interview to Match",
    "Big Contact: Preferred Communication Type", "Big Assessment Uploaded", "Big Enrollment: Created Date",
    "Big Employer", "Big Employer/School Census Block Group", "Big Approved Date", "Big Home Census Block Group", "Big Enrollment: Record Type",

    "Big Contact: Interest Finder - Entertainment","Big Contact: Interest Finder - Hobbies","Big Contact: Interest Finder - Places To Go",
    "Big Contact: Interest Finder - Sports","Little Contact: Interest Finder - Arts","Little Contact: Interest Finder - Career",
    "Little Contact: Interest Finder - Entertainment", "Little Contact: Interest Finder - Hobbies", "Little Contact: Interest Finder - Other Interests",
    "Little Contact: Interest Finder - Outdoors","Little Contact: Interest Finder - Personality","Little Contact: Interest Finder - Places To Go",
    "Little Contact: Interest Finder - Sports","Little Contact: Interest Finder - Three Wishes","Little Other Interests",
    "Little Contact: Language(s) Spoken", "Big Contact: Former Big/Little", "Big Level of Education", "Big: Military",
    "Big Languages", "Big Car Access", "Big Open to Cross-Gender Match", "Big Contact: Volunteer Availability", "Big Contact: Marital Status", "Big Re-Enroll", "Big County",
                    

    "Big ID", "Little ID", "Stage", "Closure Reason", "Closure Details", "Match Closure Meeting Date", "Big Assessment Uploaded", 
    "Little Mailing Address Census Block Group", "Little Interview Date", "Little Acceptance Date", "Little Application Received", "Little Moved to RTBM in MF",
    "Little RTBM Date in MF", "Little RTBM in Matchforce", "Little Moved to RTBM in MF", "Little Interview Date", "Little Acceptance Date", "Little RTBM in Matchforce",
]

yes_no = [
    "Big Open to Cross-Gender Match", "Big Re-Enroll", "Big Contact: Marital Status", "Big Gender", "Big Military", "Big Car Access",
    "Big Contact: Former Big/Little", 
] 

In [4]:
df.drop(to_be_deleted, axis=1, inplace=True)

In [5]:
# convert all column to lower case
df.columns = (
    df.columns.str.strip()  # Remove leading/trailing spaces
               .str.replace(':', '_', regex=True)  # Replace colons (`:`) with underscores
               .str.replace(r'\s+', '_', regex=True)  # Replace any spaces with underscores
)
df.columns = [
    x.lower().\
        replace("contact: ", "").\
        replace("finder - ", "").\
        replace("/","_").\
        replace("_18char","").\
        replace(" ", "_")
    for x in df.columns]
df.columns

Index(['match_id', 'big_age', 'big_occupation', 'big_gender', 'big_birthdate',
       'program', 'program_type', 'big_race_ethnicity',
       'match_activation_date', 'rationale_for_match', 'little_gender',
       'little_participant__race_ethnicity', 'little_birthdate',
       'match_length'],
      dtype='object')

In [6]:
# Identify categorical and datetime columns
categorical_cols = df.select_dtypes(include=['object']).columns
datetime_cols = df.select_dtypes(include=['datetime64']).columns

# Store the datetime column separately
datetime_df = df[datetime_cols]

# Drop the datetime column before imputation
df = df.drop(columns=datetime_cols)

# Convert categorical columns to 'category' dtype
for col in categorical_cols:
    df[col] = df[col].astype('category')

# Save category mappings for conversion after imputation
category_mappings = {col: dict(enumerate(df[col].cat.categories)) for col in categorical_cols}

# Convert categorical columns to numerical codes
for col in categorical_cols:
    df[col] = df[col].cat.codes
    df[col].replace(-1, np.nan, inplace=True)  # Keep missing values as NaN

# Initialize MissForest imputer
imputer = MissForest()

# Perform imputation
imputed_df = imputer.fit_transform(df)

# Convert back to dfFrame
imputed_df = pd.DataFrame(imputed_df, columns=df.columns)

# Convert categorical columns back to original categories
for col in categorical_cols:
    imputed_df[col] = imputed_df[col].round().astype(int)
    imputed_df[col] = imputed_df[col].map(category_mappings[col])

# Add the datetime column back
imputed_df = pd.concat([imputed_df, datetime_df.reset_index(drop=True)], axis=1)

100%|██████████| 5/5 [00:00<00:00, 28.04it/s]


In [7]:
imputed_df['big_race_ethnicity'].dropna(inplace=True, axis=0)
imputed_df.shape

(3275, 14)

In [8]:
data = pd.read_excel("../../data/restated/Training-Restated.xlsx")

In [9]:
sample = data[['Match ID 18Char', "Completion Date", "Match Support Contact Notes"]]
sample.rename({
    'Match ID 18Char': "match_id",
    'Completion Date':'completion_date',
    "Match Support Contact Notes":"contact_notes"
},axis=1, inplace=True)
sample.head()

,match_id,completion_date,contact_notes
0,a1v2J0000028pRvQAI,2017-11-30,Question: Activities: Answer: See ms...
1,a1v2J0000028pRvQAI,2017-10-31,Question: Activities: Answer: See MS...
2,a1v2J0000028pRvQAI,2017-12-01,Question: Activities: Answer: Match ...
3,a1v2J0000028pRvQAI,2017-05-29,Question: Activities: Answer: Msc as...
4,a1v2J0000028pRvQAI,2017-04-30,Question: Activities: Answer: Msc as...


In [10]:
sample.dropna(axis=0, inplace=True)

In [11]:
join_df = pd.merge(sample, imputed_df, how='left', on='match_id')

In [12]:
join_df.to_excel("data/cleaning.xlsx", index=False)